# Part B (Neural Network from Scratch)

You need to implement a neural network from scratch .This is a multiclass classification problem. No. of hidden layers depends on you but should be atleast 2.Remember to use activation function. You can add any other function of your choice.

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
X=X.T
y = iris.target
temp=y

In [ ]:
print(X.shape,y.shape)
print(X)

In [ ]:
print(y)

In [ ]:
def normalize_data(X):
    mean = np.mean(X, axis=1)
    std = np.std(X, axis=1)
    
    # Normalize the data
    X_normalized=[]
    for i in range(X.shape[0]):
        X_normalized.append((X[i] - np.mean(X[i])) / np.std(X[i]))
    
    return np.array(X_normalized)

In [ ]:
X=normalize_data(X)
# y=normalize_data(y)
print(X,'\n',y)

In [ ]:
def one_hot_enc(y):
    n=len(np.unique(y))
    mat=np.zeros((y.shape[0],n))
    mat[np.arange(y.shape[0]),y]=1
    return mat

In [ ]:
y=one_hot_enc(y)
print(y.shape)

In [ ]:
def parameters(layer_dimen):
  # define the parameters of your nn initially using random lib.
  param={}
  l=len(layer_dimen)-1
  for i in range(1,l+1):
    param['w'+str(i)]=np.random.randn(layer_dimen[i],layer_dimen[i-1])
    param['b'+str(i)]=np.zeros((layer_dimen[i],1))
    # print("param w is ",'\n',param['w'+str(i)],"param b is ",param['b'+str(i)],end='\n')
  return param

In [ ]:
#activation functions
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.maximum(0,x)

def softmax(x):
    x_max = np.max(x)
    exp_values = np.exp(x - x_max)
    sum_exp_values = np.sum(exp_values)
    softmax_values = exp_values / sum_exp_values
    return softmax_values


In [ ]:
def deriv_relu(x):
    return np.where(x>0,1,0)

In [ ]:
def forward(x,param,activation='relu'):
    # function for forward propagation
    L=len(param)//2
    forward_cac={}
    forward_cac['a0']=x
    for i in range(1,L):
        forward_cac['z'+str(i)]=param['w'+str(i)].dot(forward_cac['a'+str(i-1)])+param['b'+str(i)]
        forward_cac['a'+str(i)]=relu(forward_cac['z'+str(i)])
        # print("z is ",forward_cac['z'+str(i)],"a is",forward_cac['a'+str(i)],end="\n")
    
    forward_cac['z'+str(L)]=param['w'+str(L)].dot(forward_cac['a'+str(L-1)])+param['b'+str(L)]
    forward_cac['a'+str(L)]=softmax(forward_cac['z'+str(L)])
    
    return forward_cac['a'+str(L)],forward_cac


In [ ]:
def cost_funct(aL,y):
  # function for cost func if nece
  m=y.shape[1]
  cost=-(1/m)*(np.sum(y*np.log(aL)))
  return cost

In [ ]:
# use Gradient descent as of now as an optimizer

In [ ]:
def backward(al,param,forward_cac,y):
  # function for backward propagation
  grads={}
  L=len(param)//2
  m=y.shape[1]

  grads["dz"+str(L)]=al-y
  grads["dw"+str(L)]=(1/m)*np.dot(grads["dz"+str(L)],forward_cac["a"+str(L-1)].T)
  grads["db"+str(L)]=(1/m)*np.sum(grads["dz"+str(L)],axis=1,keepdims=True)

  for i in reversed(range(1,L)):
    grads["dz"+str(i)]=np.dot(param['w'+str(i+1)].T,grads["dz"+str(i+1)])*(deriv_relu(forward_cac["a"+str(i)]))
    grads["dw"+str(i)]=(1/m)*np.dot(grads["dz"+str(i)],forward_cac["a"+str(i-1)].T)
    grads["db"+str(i)]=(1/m)*np.sum(grads["dz"+str(i)],axis=1,keepdims=True)
    # print("grads of z is ",'\n',grads["dz"+str(i)],"grads of w is ",'\n',grads["dw"+str(i)],"grads of b is ",'\n',grads["db"+str(i)],end='\n')
  
  return grads

In [ ]:
def update_parameters(param,grads,lr):
#FUNCTION TO UPDATE PARAMETERS USING GD
  L=len(param)//2
  for i in range(1,L+1):
    param["w"+str(i)]=param["w"+str(i)]-(lr*grads["dw"+str(i)])
    param["b"+str(i)]=param["b"+str(i)]-(lr*grads["db"+str(i)])
    # print("param w is ",'\n',param['w'+str(i)],"param b is ",param['b'+str(i)],end='\n')
  return param

In [ ]:
def model(x,y,layer_dim,lr,iter=100):
  #function to train and build the whole model
  param=parameters(layer_dim)
  cf=[]
  for i in range(0,iter):
    al,forward_cac=forward(x,param)
    grads=backward(al,param,forward_cac,y)
    if i%10==0:
      print(al,y)
      # print(grads["dz"+str(L)])
    param=update_parameters(param,grads,lr)
    cost=cost_funct(al,y)
    cf.append(cost)
  return param,cf
  

In [ ]:
#write down the predictions and the f1 score finally
layer_dims=[X.shape[0],8,3]
lr=0.0075
iters=500
y=y.T
param,cf=model(X,y,layer_dims,lr,iters)

In [ ]:
from sklearn.metrics import accuracy_score

def predict(X, param):
    L=len(param)//2
    for i in range(1,L):
        forward_cac={}
        forward_cac['a0']=X
        for i in range(1,L):
            forward_cac['z'+str(i)]=param['w'+str(i)].dot(forward_cac['a'+str(i-1)])+param['b'+str(i)]
            forward_cac['a'+str(i)]=relu(forward_cac['z'+str(i)])
            # print("z is ",forward_cac['z'+str(i)],"a is",forward_cac['a'+str(i)],end="\n")
        
        forward_cac['z'+str(L)]=param['w'+str(L)].dot(forward_cac['a'+str(L-1)])+param['b'+str(L)]
        forward_cac['a'+str(L)]=softmax(forward_cac['z'+str(L)])
        forward_cac['a'+str(L)]  = forward_cac['a'+str(L)].T
        return np.argmax(forward_cac['a'+str(L)], axis=1)

# Predict on the test set
y_pred = predict(X,param)
print("y is",temp,"\n")
print("y pred is ",y_pred,"\n")
print(y.shape)
print(y_pred.shape)
# Calculate accuracy
accuracy = accuracy_score(temp, y_pred)
print(f"Test Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(temp, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1}")